# Dynamic Pose Integration into Static 3D Gaussian Environments using Sparse Camera Setup

### By Filip Marek Uzdowski (s220257), Jacob Schrøder Ipsen (s204440) and Rasmus Munch Kielsgaard Nielsen (s204504)

#### ATTENTION

To run the visualizer or do any training an NVIDIA graphics card with CUDA installed is required due to the rasterizer being written in CUDA. You will likely be unable to even install the dependencies if these requirements are not fulfilled. Due to this we also provided a few videos.

#### Rendering the scenes the scenes

To render and move around the scene we created a visualizer (due to the original one from the codebase was quite buggy and having insufficient features). Our pre-trained models can be downloaded from [Here]() and the original papers trained models can be gotten from here

Controls are the following

Mouse + Left Click - Rotate the camera

W - Forward

S - Backwards

A - Left

D - Right

1 - Down

2 - Up

Z - Go back 1 timestep

X - Go forward 1 timestep

T - Toggle auto play 

To close you need to stop the execution in the notebook and even then the window might persist for some time or until the notebook is closed

In [ ]:
import visualize

RENDER_COMBINED = True
# True renders both with and without background. False renders just without background
visualize.RENDER_BOTH = False

# Opens a OpenCV window with the scene.
if RENDER_COMBINED:
    # Combining scenes currently takes a while as we didn't create a great method for combining. Depending on the size of the models it can take a few minutes
    dat = visualize.get_render_data_combined_scene("params_fg.npz","max_params.npz",[0,0,0])
    
    visualize.render_scene(dat)
    
else:
    if visualize.RENDER_BOTH:
        fg,bg = visualize.get_render_data_single_scene("params_bg.npz")
        visualize.render_scene(fg,bg)
        
    else:
        data = visualize.get_render_data_single_scene("params.npz")
        visualize.render_scene(data)



### Training

This is the code for the dynamic training. The static training code has been omitted as it was more or less the same as the one found in the original paper linked here.

If you would like to train, just put the data folder in the same folder as this notebook is in.

The code below is just the train function lifted from the train.py script

In [ ]:
import torch
import os
import json
import copy
import numpy as np
from PIL import Image
from random import randint
from tqdm import tqdm
from diff_gaussian_rasterization import GaussianRasterizer as Renderer
from helpers import setup_camera, l1_loss_v1, l1_loss_v2, weighted_l2_loss_v1, weighted_l2_loss_v2, quat_mult, \
    o3d_knn, params2rendervar, params2cpu, save_params
from external import calc_ssim, calc_psnr, build_rotation, densify, update_params_and_optimizer
from train import initialize_params,initialize_optimizer,get_dataset,initialize_per_timestep,get_batch,get_loss,report_progress,initialize_post_first_timestep

def train(seq, exp):
    md = json.load(open(f"./data/{seq}/train_meta.json", 'r'))  # metadata
    num_timesteps = len(md['fn'])
    params, variables = initialize_params(seq, md)
    optimizer = initialize_optimizer(params, variables)
    output_params = []
    for t in range(num_timesteps):
        dataset = get_dataset(t, md, seq)
        todo_dataset = []
        is_initial_timestep = (t == 0)
        if not is_initial_timestep:
            params, variables = initialize_per_timestep(params, variables, optimizer)
        num_iter_per_timestep = 5000 if is_initial_timestep else 2000
        progress_bar = tqdm(range(num_iter_per_timestep), desc=f"timestep {t}")
        for i in range(num_iter_per_timestep):
            curr_data = get_batch(todo_dataset, dataset)
            loss, variables = get_loss(params, curr_data, variables, is_initial_timestep)
            loss.backward()
            with torch.no_grad():
                report_progress(params, dataset[0], i, progress_bar,opt_vars=loss)
                if is_initial_timestep:
                    pass
                    params, variables = densify(params, variables, optimizer, i)
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
        progress_bar.close()
        output_params.append(params2cpu(params, is_initial_timestep))
        if is_initial_timestep:
            variables = initialize_post_first_timestep(params, variables, optimizer)
        if (t%10 == 0 and t > 0):
            save_params(output_params, seq + str(t), exp)

train("new_jaco", "exp")
